# K-Nearest Neighbor(k-최근접이웃)
사용데이터 - 대형 매장에 방문하는 중국인과 일본인의 정보
(건물 입구부터 매장 상담원이 있는 곳까지 걸어 오는데 소요된 시간, 진열된 상품을 둘러본 횟수, 성별, 지불금액, 출신 국가)

위의 정보를 학습하여(성별 제외), 출신 국가가 블라인드된 정보만 보고 출신 국가를 예측하려 한다.

In [1]:
# 모듈 선언
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# data read
mydata = pd.read_csv('data/mydata.csv') 
del mydata['sex'] # load된 dataframe에서 성별을 제거

In [3]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
id         500 non-null int64
walk       500 non-null int64
view       500 non-null int64
payment    500 non-null int64
country    500 non-null object
dtypes: int64(4), object(1)
memory usage: 19.6+ KB


In [4]:
mydata['country'].value_counts() # 국가별 매장 방문 인원

C    300
J    200
Name: country, dtype: int64

In [5]:
# 주어진 수치값의 최대치와 최소치의 차이값으로 정규화
nomalize = lambda x : (x - min(x)) / (max(x) - min(x))

In [6]:
# 더미 코딩
mydata['country'] = (mydata['country']=='C') # china일 경우 true, japan일 경우 false

In [7]:
mydata.ix[:, 1:4].head(10) # dataframe의 walk, view, payment만 추출하여 탐색

walk  view  payment
0    17     5      176
1    12     5      194
2    32    12      384
3    25     9      329
4    23     8      290
5    24     8      246
6    22     6      235
7    32    11      353
8    30    10      324
9    13     2       93

In [8]:
mydata_n = mydata.ix[:, 1:4].apply(nomalize) # dataframe의 walk, view, payment를 이용하여 정규화
mydata_n.head(10)

walk      view   payment
0  0.240741  0.157895  0.268025
1  0.148148  0.157895  0.296238
2  0.518519  0.526316  0.594044
3  0.388889  0.368421  0.507837
4  0.351852  0.315789  0.446708
5  0.370370  0.315789  0.377743
6  0.333333  0.210526  0.360502
7  0.518519  0.473684  0.545455
8  0.481481  0.421053  0.500000
9  0.166667  0.000000  0.137931

### 실습예제
sklearn 모듈의 knn 함수를 이용하여 반지름을 30으로 설정하고, train data와 test data를 9:1로 정하여 예측해보시오.

In [9]:
feature_names = ['walk', 'view', 'payment'] # feature_name에 학습에 사용할 column name을 저장
X = mydata_n[feature_names] # feature_name에 해당하는 column만 정규화된 mydata_n에서 추출하여 X(Large x)에 저장

y = mydata['country'] # y(small y)에 mydata의 country를 저장

# 훈련 데이터와 테스트 데이터 분리
from sklearn.model_selection import train_test_split

#X와 y의 training data와 test data의 비율을 9:1로 지정(0.9, 0.1)
mydata_train, mydata_test, mydata_train_labels, mydata_test_labels = train_test_split(X, y,
                                                                                      test_size=0.1, random_state=33)

# KNN Algorism 실행
from sklearn.neighbors import KNeighborsClassifier as knn # knn모듈을 import
X = mydata_train # train (훈련 데이터중 column) => X
y = mydata_train_labels # train labels (훈련 데이터중 출신 국가) => y
mydata_test_pred = knn(n_neighbors=30) # neighbor의 수를 30으로 정함
mydata_test_pred = mydata_test_pred.fit(X, y) # data를 학습

# 분류 검증
from sklearn import metrics
y_pred = mydata_test_pred.predict(mydata_test) # test용으로 정해놓았던 data를 학습된 데이터를 이용해 출신 국가를 예측
print(metrics.classification_report(mydata_test_labels, y_pred)) # test용으로 빼 놓았던 data(출신국가 정답)과 예측값을 비교

             precision    recall  f1-score   support

      False       0.95      0.83      0.89        24
       True       0.86      0.96      0.91        26

avg / total       0.91      0.90      0.90        50

